# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df= pd.read_csv('../output_data/weather_df.csv')
weather_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Denpasar,-8.6500,115.2167,78.46,94,75,5.75,ID,1627857900
1,High Level,58.5169,-117.1360,89.71,37,20,2.30,CA,1627857811
2,Santa Maria,-29.6842,-53.8069,52.90,87,0,2.30,BR,1627857854
3,Saint George,37.1041,-113.5841,98.38,27,1,1.01,US,1627857827
4,Rafaela,-31.2503,-61.4867,57.00,38,0,6.55,AR,1627858049
...,...,...,...,...,...,...,...,...,...
573,Pangai,-19.8000,-174.3500,75.31,69,40,13.80,TO,1627858211
574,Saint-Joseph,-21.3667,55.6167,64.78,75,37,8.30,RE,1627858212
575,Constitución,-35.3333,-72.4167,54.05,68,0,8.61,CL,1627858212
576,Yar-Sale,66.8333,70.8333,54.81,97,100,16.06,RU,1627858212


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations= weather_df[['Lat','Lng']].astype(float)

# Get the humidity rating
humidity= weather_df['Humidity'].astype(float)

# Plot Heatmap, create a gmaps figure 
fig= gmaps.figure()

#create a heat layer (search the gmaps documentation - hint there's a heatmap_layer method)
heat_layer= gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=4)

#add the layer to your figure
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = weather_df[(weather_df['Max Temp'] >70) &
                      (weather_df['Max Temp'] <80) &
                      (weather_df['Wind Speed'] <10) &
                      (weather_df['Cloudiness'] ==0)
                     ]
ideal_df.dropna(inplace=True)
ideal_df

<ipython-input-5-af4799a88c43>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df.dropna(inplace=True)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,Saskylakh,71.9167,114.0833,74.61,42,0,3.47,RU,1627858052
28,Santiago,-33.4569,-70.6483,76.44,20,0,0.00,CL,1627858047
286,Tiznit Province,29.5833,-9.5000,70.83,50,0,3.56,MA,1627857908
328,Monte Patria,-30.6919,-70.9467,71.11,19,0,1.50,CL,1627858088
414,Polis,35.0367,32.4264,76.82,40,0,1.34,CY,1627858165
493,Los Andes,-32.8337,-70.5983,73.17,18,0,4.99,CL,1627857964
536,Ovalle,-30.5983,-71.2003,70.65,30,0,2.66,CL,1627858198
538,Khandyga,62.6667,135.6000,71.64,37,0,6.42,RU,1627858199


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = ideal_df
hotel_df['Hotel Name'] = ""
hotel_df

<ipython-input-6-245266804107>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ""


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,Saskylakh,71.9167,114.0833,74.61,42,0,3.47,RU,1627858052,
28,Santiago,-33.4569,-70.6483,76.44,20,0,0.00,CL,1627858047,
286,Tiznit Province,29.5833,-9.5000,70.83,50,0,3.56,MA,1627857908,
328,Monte Patria,-30.6919,-70.9467,71.11,19,0,1.50,CL,1627858088,
414,Polis,35.0367,32.4264,76.82,40,0,1.34,CY,1627858165,
493,Los Andes,-32.8337,-70.5983,73.17,18,0,4.99,CL,1627857964,
536,Ovalle,-30.5983,-71.2003,70.65,30,0,2.66,CL,1627858198,
538,Khandyga,62.6667,135.6000,71.64,37,0,6.42,RU,1627858199,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "lodging",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

#use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    
    params['location'] = f"{lat},{lng}"

    response = requests.get(base_url, params=params).json()

    results= response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    except:
        hotel_df.loc[index, "Hotel Name"] = "Hotel Not Found"
        
hotel_df

C:\Users\nick\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
14,Saskylakh,71.9167,114.0833,74.61,42,0,3.47,RU,1627858052,Hotel Not Found
28,Santiago,-33.4569,-70.6483,76.44,20,0,0.00,CL,1627858047,W Santiago
286,Tiznit Province,29.5833,-9.5000,70.83,50,0,3.56,MA,1627857908,Hotel Not Found
328,Monte Patria,-30.6919,-70.9467,71.11,19,0,1.50,CL,1627858088,Hotel Monte Cordillera
414,Polis,35.0367,32.4264,76.82,40,0,1.34,CY,1627858165,Anassa Hotel
493,Los Andes,-32.8337,-70.5983,73.17,18,0,4.99,CL,1627857964,Inca Hoteles Los Andes
536,Ovalle,-30.5983,-71.2003,70.65,30,0,2.66,CL,1627858198,Hotel Limari
538,Khandyga,62.6667,135.6000,71.64,37,0,6.42,RU,1627858199,Prezidentskaya Gostinitsa


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))